<a href="https://colab.research.google.com/github/tanucsengg/GAN/blob/main/CGAN_on_ciphar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from keras.datasets import cifar10

In [ ]:

(x1, y1), (x2, y2) = cifar10.load_data()

170508288/170498071 [==============================] - 6s 0us/step


In [ ]:
x = np.concatenate((x1, x2), axis=0)
y = np.concatenate((y1, y2), axis=0)

In [ ]:

del x1, y1, x2, y2

In [ ]:
x.shape

(60000, 32, 32, 3)

In [ ]:
y.shape

(60000, 1)

In [ ]:
pip install scipy

In [ ]:
import time
from scipy import misc
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.models import Model
from keras.layers import *
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.layers.advanced_activations import LeakyReLU

In [ ]:
img_size = 32
noise_size = 2048
batch_size = 50
classes = 10

In [ ]:
def generator():
    
    noise = Input(shape=(noise_size, ))
    label = Input(shape=(1, ))
    
    label_embedding = Flatten()(Embedding(classes, noise_size)(label))
    
    model_input = multiply([noise, label_embedding])
    
    x = Dense(2048)(model_input)
    
    x = Reshape((2, 2, 512))(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.1)(x)
    
    x = Conv2DTranspose(256, (5, 5), padding='same', strides=2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.1)(x)
    
    x = Conv2DTranspose(128, (5, 5), padding='same', strides=2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.1)(x)
    
    x = Conv2DTranspose(64, (5, 5), padding='same', strides=2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.1)(x)
    
    x = Conv2DTranspose(3, (5, 5), padding='same', strides=2)(x)
    img = Activation('tanh')(x)
    
    return Model([noise, label], img)

In [ ]:
def discriminator():
    
    img = Input(shape=(img_size, img_size, 3))
    
    x = GaussianNoise(0.1)(img)
    
    x = Conv2D(64, (3, 3), padding='same', strides = 2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.2)(x)
        
    x = Conv2D(128, (3, 3), padding='same', strides = 2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.2)(x)
        
    x = Conv2D(256, (3, 3), padding='same', strides = 2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.2)(x)
        
    x = Conv2D(512, (3, 3), padding='same', strides = 2)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU(0.2)(x)
    
    label = Input(shape=(1, ))
    label_embedding = Flatten()(Embedding(classes, noise_size)(label))
    
    flat_img = Flatten()(x)
    
    model_input = multiply([flat_img, label_embedding])

    nn = Dropout(0.3)(model_input)
    
    validity = Dense(1, activation='sigmoid')(nn)
    
    return Model([img, label], validity)

In [ ]:

d_model = discriminator()
d_model.compile(loss=['binary_crossentropy'], optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
d_model.trainable = False
g_model = generator()

noise = Input(shape=(noise_size, ))
label = Input(shape=(1, ))
img = g_model([noise, label])

valid = d_model([img, label])

combined = Model([noise, label], valid)
combined.compile(loss=['binary_crossentropy'], optimizer=Adam(lr=0.001, beta_1=0.5))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
def train(epochs):
    
    for epoch in range(epochs):
        
        random = np.random.randint(0, 11)
        
        for index in range(int(x.shape[0]/batch_size)):
                     
            valid = np.ones((batch_size, 1)) - (np.random.random()*0.1)
            fake = np.zeros((batch_size, 1)) + (np.random.random()*0.1)
            
            x_train = x[index*batch_size : (index+1)*batch_size]
            y_train = y[index*batch_size : (index+1)*batch_size]
            x_train = (x_train - 127.5)/127.5
            
            if index % 100 == random:
                valid = np.zeros((batch_size, 1)) + (np.random.random()*0.1)
                fake = np.ones((batch_size, 1)) - (np.random.random()*0.1)
            
            noise = np.random.randn(batch_size, noise_size)
            gen_img = g_model.predict([noise, y_train])
                        
            d_loss_real = d_model.train_on_batch([x_train, y_train], valid)
            d_loss_fake = d_model.train_on_batch([gen_img, y_train], fake)
            d_loss = 0.5*(np.add(d_loss_real, d_loss_fake))

            sample_label = np.random.randint(0, 10, batch_size).reshape(-1, 1)

            valid = np.ones((batch_size, 1))
            
            g_loss = combined.train_on_batch([noise, sample_label], valid)

            if index % (batch_size) == 0:
                print(index)
                print("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss, g_loss))
                sample_images(epoch)
        
        name = '/content/sample_data/weights/combined_' + str(epoch) + '.h5'
        combined.save_weights(name)
        
        time.sleep(30)

In [ ]:
def sample_images(epoch):
    r = 2
    c = 5
    noise = np.random.randn(10, noise_size)
    sample_label = np.arange(0, 10).reshape(-1, 1)
            
    gen_img = g_model.predict([noise, sample_label])
        
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            img = image.array_to_img(gen_img[cnt])
            axs[i,j].imshow(img)
            axs[i,j].set_title("Class: %d" % sample_label[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("/content/sample_data/images/%d.png" % epoch)
    plt.close()

In [ ]:
train(10)

0
0 [D loss: 0.660042] [G loss: 0.653806]
50
0 [D loss: 0.405845] [G loss: 0.427271]
100
0 [D loss: 0.260742] [G loss: 0.098891]
150
0 [D loss: 0.172595] [G loss: 0.049318]
200
0 [D loss: 0.183753] [G loss: 0.083055]
250
0 [D loss: 0.203654] [G loss: 0.047508]
300
0 [D loss: 0.188147] [G loss: 0.031626]
350
0 [D loss: 0.129009] [G loss: 0.038133]
400
0 [D loss: 0.194516] [G loss: 0.022915]
450
0 [D loss: 0.062565] [G loss: 0.031010]
500
0 [D loss: 0.297264] [G loss: 0.029245]
550
0 [D loss: 0.201348] [G loss: 0.030018]
600
0 [D loss: 0.219035] [G loss: 0.028703]
650
0 [D loss: 0.199625] [G loss: 0.038792]
700
0 [D loss: 0.142875] [G loss: 0.033770]
750
0 [D loss: 0.272500] [G loss: 0.060890]
800
0 [D loss: 0.210017] [G loss: 0.038706]
850
0 [D loss: 0.164209] [G loss: 0.069042]
900
0 [D loss: 0.209057] [G loss: 0.040888]
950
0 [D loss: 0.174913] [G loss: 0.055681]
1000
0 [D loss: 0.258968] [G loss: 0.031083]
1050
0 [D loss: 0.123873] [G loss: 0.088917]
1100
0 [D loss: 0.198831] [G loss

In [ ]:
g_model.save('/content/sample_data/gan-model')

INFO:tensorflow:Assets written to: /content/sample_data/gan-model/assets


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
import pickle

In [ ]:
pickle.dump({'train': train_history, 'test': test_history},
            open('../logs/acgan-history.pkl', 'wb'))

In [ ]:
hist = pickle.load(open('../logs/acgan-history.pkl'))

for p in ['train', 'test']:
    for g in ['discriminator', 'generator']:
        hist[p][g] = pd.DataFrame(hist[p][g], columns=['loss', 'generation_loss', 'auxiliary_loss'])
        plt.plot(hist[p][g]['generation_loss'], label='{} ({})'.format(g, p))

# get the NE and show as an equilibrium point
plt.hlines(-np.log(0.5), 0, hist[p][g]['generation_loss'].shape[0], label='Nash Equilibrium')
plt.legend()
plt.title(r'$L_s$ (generation loss) per Epoch')
plt.xlabel('Epoch')
plt.ylabel(r'$L_s$')
plt.show()